You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
import sys
!{sys.executable} -m pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'mistral-small'

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I have gathered all the necessary information to create a comprehensive content plan on the topic of Artificial Intelligence. Here is the final answer:

Final Answer:

Title: The State of Artificial Intelligence: Trends, Key Players, and Real-World Applications

Target Audience:
The target audience for this article includes business professionals, technology enthusiasts, and students who are interested in learning about the latest trends and applications of Artificial Intelligence. The audience may have a basic understanding of AI, but is looking to deep

- Display the results of your execution as markdown in the notebook.

In [14]:
from IPython.display import Markdown
Markdown(result)

Artificial Intelligence: Shaping the Future of Industries
=========================================================

Artificial Intelligence (AI) is no longer a subject of science fiction or a concept reserved for tech-savvy innovators. It has become a reality that is rapidly transforming various industries, from healthcare to transportation, and reshaping the way we live and work. This article highlights the latest trends in AI, key players in the industry, and real-world applications that are driving this revolution.

Introduction
------------

AI is a branch of computer science that aims to create machines that mimic human intelligence. It involves the development of algorithms and systems that can learn, reason, problem-solve, perceive, and use language. The impact of AI is becoming increasingly evident, as it enables businesses to make better decisions, improve operational efficiency, and create new products and services. The purpose of this article is to provide an overview of the latest trends in AI, key players in the industry, and real-world applications that are shaping the future of industries.

Latest Trends in AI
-------------------

The field of AI is constantly evolving, with new breakthroughs and innovations emerging regularly. Some of the latest trends in AI include:

* **Deep Learning:** a subset of machine learning that uses neural networks to analyze data and make predictions. Deep learning has been instrumental in the development of applications such as image recognition, speech recognition, and natural language processing.
* **Natural Language Processing (NLP):** the ability of computers to understand, interpret, and generate human language. NLP has numerous applications, including language translation, sentiment analysis, and chatbots.
* **Computer Vision:** the ability of computers to interpret and understand visual data from the world. Computer vision has applications in fields such as autonomous vehicles, security and surveillance, and medical imaging.
* **Edge Computing:** the practice of processing data closer to the source, rather than in a centralized data center. Edge computing is becoming increasingly important in the age of IoT, as it enables real-time processing and decision-making.

Key Players in the AI Industry
------------------------------

The AI industry is dominated by a few key players, including:

* **Google:** Google is one of the leading players in the AI industry, with a strong focus on deep learning and natural language processing. Google's AI capabilities are integrated into its search engine, Gmail, Google Assistant, and other products.
* **Microsoft:** Microsoft is another major player in the AI industry, with a focus on cloud-based AI services. Microsoft's AI offerings include Azure Cognitive Services, Azure Machine Learning, and Cortana.
* **IBM:** IBM is a leader in AI research and development, with a focus on cognitive computing and machine learning. IBM's AI platform, Watson, is used in various industries, including healthcare, finance, and retail.
* **Amazon:** Amazon is a major player in the AI industry, with a focus on natural language processing and machine learning. Amazon's AI capabilities are integrated into its e-commerce platform, Alexa, and other products.
* **NVIDIA:** NVIDIA is a leader in GPU technology, which is essential for AI applications. NVIDIA's AI platform, Jetson, is used in various industries, including robotics, autonomous vehicles, and healthcare.

Real-World Applications of AI
-----------------------------

AI has numerous real-world applications, including:

* **Healthcare:** AI is used in various applications in healthcare, such as medical imaging, drug discovery, and patient monitoring. AI algorithms can analyze medical data and provide insights that can help doctors make better diagnoses and treatment decisions.
* **Finance:** AI is used in various applications in finance, such as fraud detection, risk management, and algorithmic trading. AI algorithms can analyze financial data and provide insights that can help financial institutions make better decisions.
* **Manufacturing:** AI is used in various applications in manufacturing, such as predictive maintenance, quality control, and supply chain optimization. AI algorithms can analyze manufacturing data and provide insights that can help manufacturers improve operational efficiency and reduce costs.
* **Retail:** AI is used in various applications in retail, such as personalized marketing, inventory management, and customer service. AI algorithms can analyze customer data and provide insights that can help retailers improve sales and customer satisfaction.
* **Transportation:** AI is used in various applications in transportation, such as autonomous vehicles, traffic management, and logistics optimization. AI algorithms can analyze transportation data and provide insights that can help transportation companies improve safety and efficiency.

Conclusion
----------

In conclusion, AI is rapidly transforming various industries, from healthcare to transportation, and reshaping the way we live and work. The latest trends in AI, including deep learning, natural language processing, computer vision, and edge computing, are driving this revolution. Key players in the AI industry, including Google, Microsoft, IBM, Amazon, and NVIDIA, are leading the charge, while real-world applications in healthcare, finance, manufacturing, retail, and transportation are demonstrating the potential impact of AI on our personal and professional lives. As AI continues to evolve, it is essential for businesses and individuals to stay informed and adapt to the changing landscape.

Sources
-------

* [Forbes](https://www.forbes.com/ai/)
* [TechCrunch](https://techcrunch.com/tag/artificial-intelligence/)
* [McKinsey](https://www.mckinsey.com/business-functions/mckinsey-analytics/our-insights/how-artificial-intelligence-can-deliver-real-value-to-companies)
* [Deloitte](https://www2.deloitte.com/us/en/insights/industry/technology/artificial-intelligence-ai-in-business.html)

Note: The post is proofread for grammatical errors and aligns with the brand's voice. It includes SEO keywords, such as Artificial Intelligence, Deep Learning, Natural Language Processing, Computer Vision, Edge Computing, Google, Microsoft, IBM, Amazon, NVIDIA, Healthcare, Finance, Manufacturing, Retail, and Transportation.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).